In [ ]:
import pandas as pd

# 1. 시군구 CSV 파일 로드
try:
    df = pd.read_csv('/home/user/workdir/CMAQ_Emulator/main/resources/geom/si_gun_gu_allocation_v1.csv')
    print("✅ 원본 파일('si_gun_gu_allocation_v1.csv')을 성공적으로 읽었습니다.")
except FileNotFoundError:
    print("❌ 오류: 'si_gun_gu_allocation_v1.csv' 파일을 찾을 수 없습니다.")
    exit()

# 2. 모델(layers.py)의 Region Code 맵핑 정보
# '광역지자체 영문명' -> 'Region_Code' 로 변환하기 위한 역방향 맵
region_codes = {
    'Seoul City': 'A', 'Incheon City': 'B', 'Busan City': 'C', 'Daegu City': 'D',
    'Gwangju City': 'E', 'Gyeonggi-do': 'F', 'Gangwon-do': 'G', 'Chungbuk-do': 'H',
    'Chungnam-do': 'I', 'Gyeongbuk-do': 'J', 'Gyeongnam-do': 'K', 'Jeonbuk-do': 'L',
    'Jeonnam-do': 'M', 'Jeju-do': 'N', 'Daejeon City': 'O', 'Ulsan City': 'P', 'Sejong City': 'Q'
}
# '광역지자체 한글명' -> '광역지자체 영문명' 변환 맵
sido_korean_to_english = {
    '서울': 'Seoul City', '인천': 'Incheon City', '부산': 'Busan City', '대구': 'Daegu City',
    '광주': 'Gwangju City', '경기': 'Gyeonggi-do', '강원': 'Gangwon-do', '충북': 'Chungbuk-do',
    '충남': 'Chungnam-do', '경북': 'Gyeongbuk-do', '경남': 'Gyeongnam-do', '전북': 'Jeonbuk-do',
    '전남': 'Jeonnam-do', '제주': 'Jeju-do', '대전': 'Daejeon City', '울산': 'Ulsan City', '세종': 'Sejong City'
}

# 3. SIG_ENG_NM 표준화 (이전 요청 사항 반영)
mask_gu_si = (df['광역지자체'] != '서울') & (df['SIG_ENG_NM'].str.contains(','))
df.loc[mask_gu_si, 'SIG_ENG_NM'] = df.loc[mask_gu_si, 'SIG_ENG_NM'].str.split(',').str[-1].str.strip()
directional_names = ['Nam-gu', 'Buk-gu', 'Dong-gu', 'Seo-gu', 'Jung-gu']
mask_directional = df['SIG_ENG_NM'].isin(directional_names)
df.loc[mask_directional, 'SIG_ENG_NM'] = df.loc[mask_directional, 'SIG_ENG_NM'] + ', ' + df['광역지자체'].map(sido_korean_to_english)
print("✅ 시군구 영문명(SIG_ENG_NM)을 표준 형식으로 변환했습니다.")

# 4. 최종 alloc.py 형식에 맞게 데이터 구성
# 4.1. 광역지자체 영문명 컬럼 생성
df['Region_Name_Sido'] = df['광역지자체'].map(sido_korean_to_english)
# 4.2. 광역지자체 코드(A,B,C..) 컬럼 생성
df['Region_Code_Sido'] = df['Region_Name_Sido'].map(region_codes)

# 4.3. 최종 데이터프레임 선택 및 컬럼명 변경
df_for_alloc = df[['SIG_ENG_NM', 'Region_Code_Sido', 'Y(Row)', 'X(Column)', 'ratio']].copy()
df_for_alloc.columns = ['Region_Name', 'Region_Code', 'Row', 'Column', 'Ratio']

print("✅ 데이터를 새로운 alloc 형식에 맞게 재구성했습니다.")

# 5. 데이터프레임을 Python 리스트로 변환 및 파일 저장
allocation_list = df_for_alloc.values.tolist()

output_filename = 'alloc_sigungu_mapped.py'
with open(output_filename, 'w', encoding='utf-8') as f:
    f.write('# 시군구 위치정보 + 시도 Region Code 통합 버전\n')
    f.write('allocation = [\n')
    for i, item in enumerate(allocation_list):
        formatted_item = f"    [{item[0]!r}, {item[1]!r}, {item[2]}, {item[3]}, {item[4]}]"
        if i < len(allocation_list) - 1:
            formatted_item += ','
        f.write(formatted_item + '\n')
    f.write(']\n')

print(f"\n🎉 모든 변환이 완료되었습니다. 결과가 '{output_filename}' 파일로 저장되었습니다.")

In [ ]:
import pandas as pd

# 1. 시군구 CSV 파일 로드
try:
    df = pd.read_csv('si_gun_gu_allocation_v1.csv')
    print("✅ 원본 파일('si_gun_gu_allocation_v1.csv')을 성공적으로 읽었습니다.")
    print(f"✅ 확인된 컬럼명: {df.columns.tolist()}")
except FileNotFoundError:
    print("❌ 오류: 'si_gun_gu_allocation_v1.csv' 파일을 찾을 수 없습니다.")
    exit()

# 2. alloc.py 형식에 맞게 데이터 구성
# 파일에 이미 필요한 정보(Region_Code 등)가 있으므로 복잡한 변환 과정이 필요 없습니다.
# 필요한 컬럼만 순서에 맞게 선택합니다.
# 최종 Region_Name은 상세 지역명인 SIG_ENG_NM을 사용합니다.
df_for_alloc = df[['SIG_ENG_NM', 'Region_Code', 'Row', 'Column', 'Ratio']].copy()

# 최종 컬럼명을 alloc.py에서 사용하는 이름으로 변경합니다.
df_for_alloc.columns = ['Region_Name', 'Region_Code', 'Row', 'Column', 'Ratio']

print("✅ 데이터를 새로운 alloc 형식에 맞게 재구성했습니다.")

# 3. 데이터프레임을 Python 리스트로 변환 및 파일 저장
allocation_list = df_for_alloc.values.tolist()

output_filename = 'alloc_sigungu_mapped.py'
with open(output_filename, 'w', encoding='utf-8') as f:
    f.write('# 시군구 위치정보 + 시도 Region Code 통합 버전\n')
    f.write('allocation = [\n')
    for i, item in enumerate(allocation_list):
        # item[0]: Region_Name (str), item[1]: Region_Code (str)
        # item[2]: Row (int), item[3]: Column (int), item[4]: Ratio (float)
        formatted_item = f"    [{item[0]!r}, {item[1]!r}, {item[2]}, {item[3]}, {item[4]}]"
        if i < len(allocation_list) - 1:
            formatted_item += ','
        f.write(formatted_item + '\n')
    f.write(']\n')

print(f"\n🎉 모든 변환이 완료되었습니다. 결과가 '{output_filename}' 파일로 저장되었습니다.")

❌ 오류: 'si_gun_gu_allocation_v1.csv' 파일을 찾을 수 없습니다.


NameError: name 'df' is not defined

: 